In [0]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib.cm as cm
'''
1. Data
 - Config
 - Load Data
'''

# Config
minimum_value=1e-10
over_lambda=1e-5
right_lambda=1
under_lambda=10


learning_rate=0.1014
opt_threshold=0.000002748
over_theta=np.expand_dims(np.ones(100), axis=0)
right_theta=np.expand_dims(np.ones(100), axis=0)
under_theta=np.expand_dims(np.ones(100), axis=0)

#t0, t1, t2, t3, t4, t5, t6=1., 1., 1., 1., 1., 1., 1.
#t7, t8, t9, t10, t11, t12, t13=1., 1., 1., 1., 1., 1., 1.
#theta=np.array([[t0, t1, t2, t3, t4, t5, t6,
#                 t7, t8, t9, t10, t11, t12, t13]])


# Load data
path = "drive/My Drive/Classroom/Machine Learning (2) 2020-1\
/class-MachineLearning/assignment07/"
train=[]
test=[]

data = np.genfromtxt(path+"data-nonlinear.txt", delimiter=',')

pointX  = data[:, 0]
pointY  = data[:, 1]

label   = data[:, 2]
m=len(pointX)
pointX0 = pointX[label == 0]
pointY0 = pointY[label == 0]

pointX1 = pointX[label == 1]
pointY1 = pointY[label == 1]


'''
#x2=pointX**2
#y2=pointY**2
x1=pointX
y1=pointY

x3=pointX**3
y3=pointY**3

x2=pointX**2
y2=pointY**2
x4=pointX**4
y4=pointY**4
xy=pointX*pointY
x3y2=pointX**3*(pointY**2)
x2y3=pointX**2*(pointY**3)
x5=pointX**5
y5=pointY**5
f=[x1, y1, x3, y3, x2, y2, x4, y4, xy, x3y2, x2y3, x5, y5]


for i in f :
    input_data=np.hstack((input_data, i.reshape(-1, 1)))
input_data=temp.reshape(-1, 1)
'''

for i in range(10) :
    for j in range(10) :
        if i==j==0 :
            input_data=(pointX**i * (pointY**j)).reshape(-1, 1)
            continue
        t=(pointX**i) * (pointY**j)
        input_data=np.hstack((input_data, t.reshape(-1, 1)))
print(input_data)
print(input_data.shape)

In [0]:
'''
Visualize Data
'''
plt.figure()
plt.scatter(pointX0, pointY0, c='b')
plt.scatter(pointX1, pointY1, c='r')
plt.tight_layout()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [0]:
print(len(theta[0]))
print(input_data[ :, 0])
print(input_data[ :, 1])
t=[1 for i in range(100)]
theta[0] = t
print(type(theta))
print(type(theta[0]))

In [0]:
X=np.array([1 for i in range(100)])
for i in range(10) :
    for j in range(10) :
        X[10*i + j] = 10*i + j
print(X)
X[10]=100
X[11]=111
print(X)

In [0]:
'''
2. Hypothesis
 - Define hypothesis and sigmoid hypothesis.

3. Objective function
 - Define objective function.

4. Gradient Descent
 - Define derivations.
 - Update thetas
'''

# Define hypothesis
def hypothesis() :
    return (input_data.dot(over_theta.T).flatten(), \
            input_data.dot(right_theta.T).flatten(), \
            input_data.dot(under_theta.T).flatten())

def sigmoid_h() :
    return (1/(1+np.exp(-over_z)), 1/(1+np.exp(-right_z)), 1/(1+np.exp(-under_z)))
    
over_z, right_z, under_z=hypothesis()
over_h, right_h, under_h=sigmoid_h()

# Define objective function

def objective_function() :
    #positive_z=np.array(list(map(lambda x : max(x, 0), z)))
    #J = np.sum(positive_z - z*label + np.log(1+np.exp(-np.abs(z)))) / m + over_lambda/2 * (np.sum(theta**2))
    over_J = np.sum(-label*np.log(over_h+minimum_value) - (1-label)*np.log(1-over_h+minimum_value)) / m + over_lambda/2 * (np.sum(over_theta**2))
    right_J = np.sum(-label*np.log(right_h+minimum_value) - (1-label)*np.log(1-right_h+minimum_value)) / m + right_lambda/2 * (np.sum(right_theta**2))
    under_J = np.sum(-label*np.log(under_h+minimum_value) - (1-label)*np.log(1-under_h+minimum_value)) / m + under_lambda/2 * (np.sum(under_theta**2))
    return over_J, right_J, under_J

# Define derivations
def derivate() :    
    global over_dev, right_dev, under_dev
    for i in range(len(over_theta[0])) :
        over_dev[i] = np.sum((over_h-label) * input_data[:,i]) / m + (over_lambda * over_theta[0][i])
    for i in range(len(over_theta[0])) :
        right_dev[i] = np.sum((right_h-label) * input_data[:,i]) / m + (right_lambda * right_theta[0][i])
    for i in range(len(over_theta[0])) :
        under_dev[i] = np.sum((under_h-label) * input_data[:,i]) / m + (under_lambda * under_theta[0][i])
        
    '''
    dev0=np.sum(h-label) / m
    dev1=np.sum((h-label)*x1) / m
    dev2=np.sum((h-label)*y1) / m
    
    dev3=np.sum((h-label)*x3) / m
    dev4=np.sum((h-label)*y3) / m
    
    dev5=np.sum((h-label)*x2) / m
    dev6=np.sum((h-label)*y2) / m
    dev7=np.sum((h-label)*x4) / m
    dev8=np.sum((h-label)*y4) / m
    dev9=np.sum((h-label)*xy) / m
    dev10=np.sum((h-label)*x3y2) / m
    dev11=np.sum((h-label)*x2y3) / m
    dev12=np.sum((h-label)*x5) / m
    dev13=np.sum((h-label)*y5) / m
    
    dev=[dev0, dev1, dev2, dev3, dev4, dev5, dev6, dev7, dev8,
         dev9, dev10, dev11, dev12, dev13]
    '''

# Update thetas
def gradient_descent() :
    global over_theta, right_theta, under_theta
    for i in range(len(over_theta[0])) :
        over_theta[0][i] = over_theta[0][i] - learning_rate * over_dev[i]
    for i in range(len(over_theta[0])) :
        right_theta[0][i] = right_theta[0][i] - learning_rate * right_dev[i]
    for i in range(len(over_theta[0])) :
        under_theta[0][i] = under_theta[0][i] - learning_rate * under_dev[i]
    '''
    return (theta[0][0]-(learning_rate*dev0), theta[0][1]-(learning_rate*dev1),\
            theta[0][2]-(learning_rate*dev2), theta[0][3]-(learning_rate*dev3),\
            theta[0][4]-(learning_rate*dev4), theta[0][5]-(learning_rate*dev5),\
            theta[0][6]-(learning_rate*dev6), theta[0][7]-(learning_rate*dev7),\
            theta[0][8]-(learning_rate*dev8), theta[0][9]-(learning_rate*dev9),\
            theta[0][10]-(learning_rate*dev10), theta[0][11]-(learning_rate*dev11),\
            theta[0][12]-(learning_rate*dev12), theta[0][13]-(learning_rate*dev13))
    '''
def accuracy() :
    over_p=np.array(over_h>=0.5, dtype=np.float64)
    over_c=np.array(label==over_p, dtype=np.float64)
    right_p=np.array(right_h>=0.5, dtype=np.float64)
    right_c=np.array(label==right_p, dtype=np.float64)
    under_p=np.array(under_h>=0.5, dtype=np.float64)
    under_c=np.array(label==under_p, dtype=np.float64)
    return (np.sum(over_c)/m, np.sum(right_c)/m, np.sum(under_c)/m)

over_J_list=[]
right_J_list=[]
under_J_list=[]
over_accuracy_list=[]
right_accuracy_list=[]
under_accuracy_list=[]
over_h_list=[]
right_h_list=[]
under_h_list=[]

over_dev=[0 for i in range(len(over_theta[0]))]
right_dev=[0 for i in range(len(over_theta[0]))]
under_dev=[0 for i in range(len(over_theta[0]))]

# Train
count=0
for i in range(1000) :
    
    over_h_list.append(over_h)
    right_h_list.append(right_h)
    under_h_list.append(under_h)
    #if i>=10000 and accuracy() > accuracy_list[-1] :
    #    learning_rate=0.00001
    over_a, right_a, under_a = accuracy()
    over_accuracy_list.append(over_a)
    right_accuracy_list.append(right_a)
    under_accuracy_list.append(under_a)
    count+=1
    overj, rightj, underj=objective_function()
    over_J_list.append(overj)
    right_J_list.append(rightj)
    under_J_list.append(underj)
    
    derivate()
    '''
    dev0, dev1, dev2, dev3, dev4, dev5, dev6, dev7, dev8,\
    dev9, dev10, dev11, dev12, dev13 = dev()
    
    theta[0][0], theta[0][1], theta[0][2], theta[0][3], theta[0][4],\
    theta[0][5], theta[0][6], theta[0][7], theta[0][8], theta[0][9],\
    theta[0][10], theta[0][11], theta[0][12], theta[0][13] = gradient_descent()
    '''
    gradient_descent()
    over_z, right_z, under_z=hypothesis()
    over_h, right_h, under_h=sigmoid_h()




X = np.array([1. for i in range(len(over_theta[0]))])
def H(x, y) :
    global X
    #X=[]
    for i in range(10) :
        for j in range(10) :
            X[10*i + j]=(x**i) * (y**j)
    
    over_H=X.dot(over_theta.T)
    right_H=X.dot(right_theta.T)
    under_H=X.dot(under_theta.T)
    over_grid_H.append(1/(1+np.exp(-over_H)))
    right_grid_H.append(1/(1+np.exp(-right_H)))
    under_grid_H.append(1/(1+np.exp(-under_H)))

'''
For plot contour
'''
gridx=np.linspace(start=-1, stop=1, num=101)
gridy=np.linspace(start=-1, stop=1, num=101)
# Construct meshgrid between x and y
grid_x, grid_y=np.meshgrid(gridx, gridy)      

over_grid_H=[]
right_grid_H=[]
under_grid_H=[]


for (x, y) in zip(np.ravel(grid_x), np.ravel(grid_y)) :
    H(x, y)

#print(over_grid_H)
over_grid_H=np.array(over_grid_H).reshape(grid_x.shape)
right_grid_H=np.array(right_grid_H).reshape(grid_x.shape)
under_grid_H=np.array(under_grid_H).reshape(grid_x.shape)
#print(over_grid_H)
#H(x, y) for (x, y) in \
                 #zip(np.ravel(grid_x), np.ravel(grid_y))

#grid_H = grid_H.reshape(grid_x.shape)

#print(J_list)

In [0]:
'''
Visualize Loss
'''
plt.figure(figsize=(10, 7))
plt.plot(over_J_list, color='red', label='over-fitting')
plt.plot(right_J_list, color='green', label='just-right')
plt.plot(under_J_list, color='blue', label='under-fitting')
plt.xlabel("iteration")
plt.ylabel("cost")
plt.legend(loc='upper right')
plt.title("Energy values (train)")
plt.show()
#print(over_J_list)
#print(right_J_list)
#print(under_J_list)

In [0]:
print(over_J_list[-10:])
print(right_J_list[-10:])
print(under_J_list[-10:])
'''
Visualize accuracy
'''
plt.figure(figsize=(10, 7))
plt.plot(over_accuracy_list, color='red', label='over-fitting')
plt.plot(right_accuracy_list, color='green', label='just-right')
plt.plot(under_accuracy_list, color='blue', label='under-fitting')
plt.xlabel("iteration")
plt.ylabel("accuracy")
plt.legend(loc='upper right')
plt.title("Accuracy")
plt.show()

In [0]:
print("<Final training accuracy>")
print('-'*30)
print(over_accuracy_list[-1]*100, "%")
print(right_accuracy_list[-1]*100, "%")
print(under_accuracy_list[-1]*100, "%")
print('-'*30)

In [0]:
'''
Visualize classifier
'''
print(over_grid_H)
fig = plt.figure(figsize=(10, 7))
plt.contour(grid_x, grid_y, over_grid_H, levels=[0.5], colors='red')
plt.contour(grid_x, grid_y, np.array(right_grid_H), levels=[0.5], colors='green')
plt.contour(grid_x, grid_y, np.array(under_grid_H), levels=[0.5], colors='blue')
plt.scatter(pointX0, pointY0, alpha=1, c='b', s=5)
plt.scatter(pointX1, pointY1, alpha=1, c='r', s=5)
plt.show()